In [1]:
import mysql.connector
import requests
import collections
import datetime
import pandas as pd

In [ ]:
conn_relacional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="relacional",
)

conn_dimensional = mysql.connector.connect(
    host="127.0.0.1",
    port=3306,
    user="root",
    password="0101",
    database="dimensional",
)

cursor_relacional = conn_relacional.cursor()
cursor_dimensional = conn_dimensional.cursor()

In [ ]:
def pais_para_continente(pais):
    url = f"https://restcountries.com/v3.1/translation/{pais.lower()}"
    response = requests.get(url)

    if response.status_code != 200:
        return None
    
    data = response.json()[0]
    continente = data['region']

    traducao = {
        'Africa': 'África',
        'Americas': 'América',
        'Europe': 'Europa',
        'Asia': 'Ásia',
        'Oceania': 'Oceania',
        'Antarctic': 'Antártida'
    }
    return traducao.get(continente, continente)

## Data

In [ ]:
# Leitura do xls
excel_path = 'Ch3-SampleDateDim.xls'
df = pd.read_excel(excel_path)

In [ ]:
for i, row in df.iterrows():
    day = row['day num in month']
    month = row['month']
    year = row['year']
    quarter = row['quarter']
    semester = quarter // 3 + 1
    
    cursor_dimensional.execute(
        "INSERT INTO d_data (idData, dia, mes, ano, trimestre, semestre) VALUES (%s, %s, %s, %s, %s, %s)",
        (i, day, month, year, quarter, semester)
    )

## Localidade

In [ ]:
cursor_relacional.execute(
    "SELECT idPais, nmPais FROM pais"
)
paises = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_localidade (idLocalidade, pais, continente) VALUES (%s, %s, %s)",
    [(pais[0], pais[1], pais_para_continente(pais[1])) for pais in paises],
)

## Plano

In [ ]:
cursor_relacional.execute(
    "SELECT idPlano, nmPlano FROM plano",
)
planos = cursor_relacional.fetchall()

cursor_dimensional.executemany(
    "INSERT INTO d_plano (idPlano, nmPlano) VALUES (%s, %s)",
    planos
)

## Assinatura

In [ ]:
data_lookup = {}
cursor_dimensional.execute("SELECT idData, ano, mes FROM d_data WHERE dia = 1")
for id_data, ano, mes in cursor_dimensional.fetchall():
    data_lookup[(ano, mes)] = id_data

localidade_lookup = {}
cursor_dimensional.execute("SELECT idLocalidade, pais, continente FROM d_localidade")
for id_localidade, pais, continente in cursor_dimensional.fetchall():
    localidade_lookup[(pais, continente)] = id_localidade

cursor_relacional.execute(
    """
    SELECT
        u.dtNascimento,
        p.nmPais,
        a.dtInicio
    FROM assinatura a
    JOIN usuario u ON u.idUsuario = a.Usuario_idUsuario
    JOIN pais p ON p.idPais = u.pais_idPais
    """
)
assinaturas = cursor_relacional.fetchall()

assinaturas_agregadas = collections.defaultdict(int)
data_atual = datetime.datetime(2018, 12, 31) # Considerando o final da planilha

for dt_nascimento, pais, dt_inicio in assinaturas:
    idade_usuario = data_atual.year - dt_nascimento.year - \
        ((data_atual.month, data_atual.day) < (dt_nascimento.month, dt_nascimento.day))
    faixa_etaria_usuario = idade_usuario // 10

    continente = pais_para_continente(pais)
    id_localidade = localidade_lookup.get((pais, continente))

    id_data = data_lookup.get((dt_inicio.year, dt_inicio.month))

    assinaturas_agregadas[(id_data, id_localidade, faixa_etaria_usuario)] += 1

cursor_dimensional.executemany(
    'INSERT INTO f_assinatura (idAssinatura, numAssinantes, faixa_etaria, d_Localidade_idLocalidade, d_Data_idData) VALUES (%s, %s, %s, %s, %s)',
    [
        (i, num_assinaturas, faixa_etaria, id_localidade, id_data)
        for i, ((id_data, id_localidade, faixa_etaria), num_assinaturas) in enumerate(assinaturas_agregadas.items(), start=1)
    ]
)

## Receita

In [ ]:
data_lookup = {}
cursor_dimensional.execute("SELECT idData, ano, mes, trimestre FROM d_data WHERE dia = 1")
for id_data, ano, mes, trimestre in cursor_dimensional.fetchall():
    data_lookup[(ano, mes)] = id_data

cursor_relacional.execute("""
    SELECT 
        p.dtPagamento,
        p.vlrRecebido,
        a.Plano_idPlano
    FROM pagamento p
    JOIN assinatura a ON a.idAssinatura = p.Assinatura_idAssinatura    
""")
pagamentos = cursor_relacional.fetchall()
receita_agregada = collections.defaultdict(float)

for dt_pagamento, vlr_recebido, id_plano in pagamentos:
    dt_pagamento = datetime.datetime.strptime(dt_pagamento, "%Y-%m-%d")

    ano_pagamento = dt_pagamento.year
    mes_pagamento = dt_pagamento.month
    id_data = data_lookup.get((ano_pagamento, mes_pagamento))
    
    receita_agregada[(id_data, id_plano)] += float(vlr_recebido)

cursor_dimensional.executemany(
    'INSERT INTO f_receita (idReceita, valorReceita, d_Plano_idPlano, d_Data_idData) VALUES (%s, %s, %s, %s)',
    [
        (i, valor_receita, id_plano, id_data)
        for i, ((id_data, id_plano), valor_receita) in enumerate(receita_agregada.items(), start=1)
    ]
)

In [ ]:
conn_dimensional.commit()